First, we need to add the current folder to the PYTHONPATH in the code with the absolute location for the torchMPS library!

In [26]:
import sys
sys.path.append(r"C:\Users\piche\Documents\dereck_codebase\udem_code\ift_4055\honor_project\src\TorchMPS-main")

In [ ]:
import torch 
import torch.nn as nn
from torchmps import MPS

Now, let us first build a fully connected simple layer by layer approach to the procedure. 

In [ ]:
class MLP(nn.module):
    def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 6, 5)
    self.conv2 = nn.Conv2d(6, 16, 5)
    # an affine operation: y = Wx + b
    self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

    
